# Unit Testing Code in Python
## A very brief overview
Jörg Dietrich

# Why Unit Testing and What Is It?

* Automatically test small units (functions, methods) of your code.
* Remove necessity to run large programs end-to-end to test small changes
* Catch regressions before they are shipped to other users/used in your research
* Modular, no need to test everything, everytime
* Ideally, large parts of the code should be covered by tests, covering regular and corner cases

# Several Frameworks

* nose (not developed anymore)
* nose2 (successor of nose)
* unittest in the Python Standard Library
* pytest (this presentation)

# Example Code

In [1]:
%%writefile cosmology.py
import numpy as np
from scipy.integrate import quad


class Cosmology:
    def __init__(self, omega_m=0.3, omega_l=0.7, h=0.7):
        self.omega_m = omega_m
        self.omega_l = omega_l
        self.omega_k = 1. - self.omega_m - self.omega_l
        self.h = h
        self.dh = 3000 / self.h   # Hubble distance in Mpc.
        return

    def Ez(self, z):
        e = np.sqrt(self.omega_m * (1 + z)**3 + self.omega_k * (1 + z)**2
                    + self.omega_l)
        return e

    def ooEz(self, z):
        """Returns 1/E(z)"""
        return 1 / self.Ez(z)

    def comoving_line_of_sight_distance(self, z1, z2):
        """Returns the line of sight comoving distance between objects at
        redshifts z1 and z2, z2>z1. Value is in Mpc/h"""
        if z1 >= z2:
            raise ValueError("z2 must be greater than z1")
        dclos = self.dh * quad(self.ooEz, z1, z2)[0]
        return dclos

    def angular_diameter_distance(self, z1, z2):
        raise NotImplementedError("not yet")

Writing cosmology.py


# Writing Our First Tests

* Every file starting with test\_ will be inspected
* Every function starting with test\_ will be executed

when running pytest.

In [2]:
%%writefile test_cosmology.py
from cosmology import Cosmology

from numpy.testing import assert_almost_equal


def test_ez():
    cosmo = Cosmology()
    actual = cosmo.Ez(1)
    assert_almost_equal(actual, 1.7606816861659007)
    
def test_comoving_line_of_sight_distance():
    cosmo = Cosmology()
    actual = cosmo.comoving_line_of_sight_distance(0, 1)
    assert_almost_equal(actual, 3306.1159989763337)

Writing test_cosmology.py


In [3]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/joerg/TALKS/2018-05-08_code-coffee, inifile:
plugins: remotedata-0.2.0, openfiles-0.2.0, doctestplus-0.1.2, cov-2.5.1, arraydiff-0.2, hypothesis-3.38.5
collected 2 items                                                              

test_cosmology.py ..                                                     [100%]

=========================== 2 passed in 0.09 seconds ===========================


# Testing Exceptions

In [4]:
%%writefile test_cosmology.py
from cosmology import Cosmology

import pytest

from numpy.testing import assert_almost_equal


def test_ez():
    cosmo = Cosmology()
    actual = cosmo.Ez(1)
    assert_almost_equal(actual, 1.7606816861659007)


def test_comoving_line_of_sight_distance():
    cosmo = Cosmology()
    actual = cosmo.comoving_line_of_sight_distance(0, 1)
    assert_almost_equal(actual, 3306.1159989763337)


def test_comoving_line_of_sight_distance_exception():
    cosmo = Cosmology()
    with pytest.raises(ValueError) as err:
        cosmo.comoving_line_of_sight_distance(1, 0)
    assert "z2 must be greater than z1" in str(err.value)

Overwriting test_cosmology.py


In [5]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/joerg/TALKS/2018-05-08_code-coffee, inifile:
plugins: remotedata-0.2.0, openfiles-0.2.0, doctestplus-0.1.2, cov-2.5.1, arraydiff-0.2, hypothesis-3.38.5
collected 3 items                                                              

test_cosmology.py ...                                                    [100%]

=========================== 3 passed in 0.08 seconds ===========================


# Markers

Markers are decorators, which can be used to tell `pytest`

* to skip a test (`@pytest.mark.skip`)
* conditionally skip a test (`@pytest.mark.skipif`)
* parametrize a test (run with varying input) (`@pytest.mark.parametrize`)
* a test is expected to fail (`@pytest.mark.xfail`)

Custom markers can be defined.

In [6]:
%%writefile test_cosmology.py
from cosmology import Cosmology

import pytest

from numpy.testing import assert_almost_equal


@pytest.mark.parametrize('z, expected', [(0.1, 1.048475083156486),
                                         (0.5, 1.3086252328302401),
                                         (1.0, 1.7606816861659007)])
def test_ez(z, expected):
    cosmo = Cosmology()
    actual = cosmo.Ez(z)
    assert_almost_equal(actual, expected)


def test_comoving_line_of_sight_distance():
    cosmo = Cosmology()
    actual = cosmo.comoving_line_of_sight_distance(0, 1)
    assert_almost_equal(actual, 3306.1159989763337)


def test_comoving_line_of_sight_distance_exception():
    cosmo = Cosmology()
    with pytest.raises(ValueError) as err:
        cosmo.comoving_line_of_sight_distance(1, 0)
    assert "z2 must be greater than z1" in str(err.value)

Overwriting test_cosmology.py


In [7]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/joerg/TALKS/2018-05-08_code-coffee, inifile:
plugins: remotedata-0.2.0, openfiles-0.2.0, doctestplus-0.1.2, cov-2.5.1, arraydiff-0.2, hypothesis-3.38.5
collected 5 items                                                              

test_cosmology.py .....                                                  [100%]

=========================== 5 passed in 0.09 seconds ===========================


In [8]:
%%writefile test_cosmology.py
from cosmology import Cosmology

import pytest

from numpy.testing import assert_almost_equal


@pytest.mark.xfail
def test_angular_diameter_distance():
    cosmo = Cosmology()
    actual = cosmo.angular_diameter_distance(0, 1)
    desired = 1156.34008206
    assert_almost_equal(actual, desired)

@pytest.mark.parametrize('z, expected', [(0.1, 1.048475083156486),
                                         (0.5, 1.3086252328302401),
                                         (1.0, 1.7606816861659007)])
def test_ez(z, expected):
    cosmo = Cosmology()
    actual = cosmo.Ez(z)
    assert_almost_equal(actual, expected)


def test_comoving_line_of_sight_distance():
    cosmo = Cosmology()
    actual = cosmo.comoving_line_of_sight_distance(0, 1)
    assert_almost_equal(actual, 3306.1159989763337)


def test_comoving_line_of_sight_distance_exception():
    cosmo = Cosmology()
    with pytest.raises(ValueError) as err:
        cosmo.comoving_line_of_sight_distance(1, 0)
    assert "z2 must be greater than z1" in str(err.value)

Overwriting test_cosmology.py


In [9]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/joerg/TALKS/2018-05-08_code-coffee, inifile:
plugins: remotedata-0.2.0, openfiles-0.2.0, doctestplus-0.1.2, cov-2.5.1, arraydiff-0.2, hypothesis-3.38.5
collected 6 items                                                              

test_cosmology.py x.....                                                 [100%]

===================== 5 passed, 1 xfailed in 0.14 seconds ======================


# Classes as containers for tests

We had to instantiate `Cosmology` every time we defined a new test. Wrapping all tests in a class makes this less repetitive.

In [10]:
%%writefile test_cosmology_class.py
from cosmology import Cosmology

import pytest

from numpy.testing import assert_almost_equal


class TestCosmology:
    def setup_class(cls):
        cls.cosmo = Cosmology()

    @pytest.mark.parametrize('z, expected', [(0.1, 1.048475083156486),
                                         (0.5, 1.3086252328302401),
                                         (1.0, 1.7606816861659007)])
    def test_ez(self, z, expected):
        actual = self.cosmo.Ez(1)
        assert_almost_equal(actual, 1.7606816861659007)

    def test_comoving_line_of_sight_distance(self):
        actual = self.cosmo.comoving_line_of_sight_distance(0, 1)
        assert_almost_equal(actual, 3306.1159989763337)

    def test_comoving_line_of_sight_distance_exception(self):
        with pytest.raises(ValueError) as err:
            self.cosmo.comoving_line_of_sight_distance(1, 0)
        assert "z2 must be greater than z1" in str(err.value)

    @pytest.mark.xfail
    def test_angular_diameter_distance(self):
        actual = self.cosmo.angular_diameter_distance(0, 1)
        desired = 1156.34008206
        assert_almost_equal(actual, desired)

Writing test_cosmology_class.py


In [11]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/joerg/TALKS/2018-05-08_code-coffee, inifile:
plugins: remotedata-0.2.0, openfiles-0.2.0, doctestplus-0.1.2, cov-2.5.1, arraydiff-0.2, hypothesis-3.38.5
collected 12 items                                                             

test_cosmology.py x.....                                                 [ 50%]
test_cosmology_class.py .....x                                           [100%]

===================== 10 passed, 2 xfailed in 0.16 seconds =====================


# Suggested Work Flow

* For every new method or function write a unit test
* Try to cover normal input as well as corner cases
* If an Exception can be raised, test for that as well
* If you find a bug fix it and then add a test to ensure that the bug stays fixed

## Example:

In [12]:
%%writefile fib.py
def fibonacci(n):
    """Compute the n-th Fibonacci number by iteration"""
    if n in [0, 1]:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

Writing fib.py


In [13]:
%%writefile test_fib.py
from numpy.testing import assert_array_equal
import numpy as np

import pytest

from fib import fibonacci


def test_fibonacci():
    actual = np.array([fibonacci(i) for i in range(10)])
    desired = np.array([0, 1, 1, 2, 3, 5, 8, 13, 21, 34])
    assert_array_equal(actual, desired)


Writing test_fib.py


In [14]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/joerg/TALKS/2018-05-08_code-coffee, inifile:
plugins: remotedata-0.2.0, openfiles-0.2.0, doctestplus-0.1.2, cov-2.5.1, arraydiff-0.2, hypothesis-3.38.5
collected 13 items                                                             

test_cosmology.py x.....                                                 [ 46%]
test_cosmology_class.py .....x                                           [ 92%]
test_fib.py .                                                            [100%]

===================== 11 passed, 2 xfailed in 0.17 seconds =====================


### But there is a problem:

In [15]:
from IPython.core.magic import register_cell_magic
import sys

@register_cell_magic
def handle(line, cell):
    try:
        exec(cell)
    except Exception as e:
        print(sys.exc_info())

In [16]:
%%handle
from fib import fibonacci

fibonacci(9.5)

(<class 'RecursionError'>, RecursionError('maximum recursion depth exceeded in comparison',), <traceback object at 0x7f7f916e2e48>)


In [17]:
%%writefile fib.py
def fibonacci(n):
    """Compute the n-th Fibonacci number by iteration"""
    if n != int(n):
        raise ValueError("n must be integer")
    if n in [0, 1]:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

Overwriting fib.py


In [18]:
%%writefile test_fib.py
from numpy.testing import assert_array_equal
import numpy as np

import pytest

from fib import fibonacci


def test_fibonacci():
    actual = np.array([fibonacci(i) for i in range(10)])
    desired = np.array([0, 1, 1, 2, 3, 5, 8, 13, 21, 34])
    assert_array_equal(actual, desired)

def test_fibonacci_int():
    with pytest.raises(ValueError):
        fibonacci(9.5)

Overwriting test_fib.py


In [19]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/joerg/TALKS/2018-05-08_code-coffee, inifile:
plugins: remotedata-0.2.0, openfiles-0.2.0, doctestplus-0.1.2, cov-2.5.1, arraydiff-0.2, hypothesis-3.38.5
collected 14 items                                                             

test_cosmology.py x.....                                                 [ 42%]
test_cosmology_class.py .....x                                           [ 85%]
test_fib.py ..                                                           [100%]

===================== 12 passed, 2 xfailed in 0.16 seconds =====================


# Bonus: Custom decorator

In [20]:
%%writefile test_fib.py
from numpy.testing import assert_array_equal
import numpy as np

import pytest

from fib import fibonacci


def test_fibonacci():
    actual = np.array([fibonacci(i) for i in range(10)])
    desired = np.array([0, 1, 1, 2, 3, 5, 8, 13, 21, 34])
    assert_array_equal(actual, desired)

def test_fibonacci_int():
    with pytest.raises(ValueError):
        fibonacci(9.5)
        
@pytest.mark.slow
def test_fibonacci_large():
    assert fibonacci(36) == 14930352

Overwriting test_fib.py


In [21]:
%%writefile conftest.py
import pytest


def pytest_addoption(parser):
    parser.addoption("--runslow", action="store_true", help="run slow tests")


def pytest_runtest_setup(item):
    if 'slow' in item.keywords and not item.config.getoption("--runslow"):
        pytest.skip("need --runslow option to run")

Writing conftest.py


In [22]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/joerg/TALKS/2018-05-08_code-coffee, inifile:
plugins: remotedata-0.2.0, openfiles-0.2.0, doctestplus-0.1.2, cov-2.5.1, arraydiff-0.2, hypothesis-3.38.5
collected 15 items                                                             

test_cosmology.py x.....                                                 [ 40%]
test_cosmology_class.py .....x                                           [ 80%]
test_fib.py ..s                                                          [100%]

=============== 12 passed, 1 skipped, 2 xfailed in 0.15 seconds ================


In [23]:
!pytest --runslow

============================= test session starts ==============================
platform linux -- Python 3.6.4, pytest-3.3.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/joerg/TALKS/2018-05-08_code-coffee, inifile:
plugins: remotedata-0.2.0, openfiles-0.2.0, doctestplus-0.1.2, cov-2.5.1, arraydiff-0.2, hypothesis-3.38.5
collected 15 items                                                             

test_cosmology.py x.....                                                 [ 40%]
test_cosmology_class.py .....x                                           [ 80%]
test_fib.py ...                                                          [100%]

==================== 13 passed, 2 xfailed in 14.48 seconds =====================
